In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
386596,4.031000e+09,4,403,1,40310001,1,4,C,2.0,1.0,2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
254721,2.132001e+09,2,213,2,21320014,2,4,C,NaN,NaN,1,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
340446,3.132000e+09,3,313,2,31320003,12,10,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN
428641,4.192002e+09,4,419,2,41920016,13,1,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309411,3.052002e+09,3,305,2,30520022,16,3,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,4.0,14.0


In [3]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [4]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [5]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [6]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
85741,1.152002e+09,punjab,hafiza abad,rural,11520017,B,3,male,present,son/daughter,1988,0,0,18,unmarried
293896,2.151001e+09,sindh,thaeparkar,urban,21510012,B,2,female,present,wife/husband,1966,0,0,40,married
116596,1.192003e+09,punjab,kasur,rural,11920025,B,2,female,present,wife/husband,1980,0,0,26,married
450206,4.092002e+09,balochistan,kalat,rural,40920015,B,3,male,present,son/daughter,1994,0,0,12,unmarried
482751,4.222001e+09,balochistan,qillah siafullah,rural,42220007,B,2,female,present,wife/husband,1982,0,0,23,married


In [7]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [8]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [10]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [11]:
# Select people in school enrollment age (I would say 4-10 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 15)]

In [12]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    66049
2    42560
3    34792
4    29827
Name: province, dtype: int64

In [13]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")

In [14]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
94,Balochistan,Kohlu,40720
1,Punjab,Sargodha,10511
10,Sindh,Hyderabad,21011
13,Balochistan,Quetta,40111
106,Balochistan,Loralai,41920


In [17]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
22,Punjab,Faisalabad,10920,1092
36,Punjab,Sahiwal,12320,1232
31,Punjab,Lahore,11820,1182
110,Balochistan,Nasirabad,42320,4232
48,Sindh,Khairpur,20120,2012


In [18]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          144401
left_only          0
right_only         0
Name: _merge, dtype: int64

In [19]:
years06_07_

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
0,1001100103.0,5,15,female,unmarried,1,10.0,2.0,5.0,urban,1,1001,Punjab,Islamabad,10011,both
1,1001100103.0,6,13,male,unmarried,1,6.0,1.0,NaN,urban,1,1001,Punjab,Islamabad,10011,both
2,1001100104.0,4,15,female,unmarried,1,5.0,2.0,5.0,urban,1,1001,Punjab,Islamabad,10011,both
3,1001100104.0,5,13,female,unmarried,1,5.0,2.0,5.0,urban,1,1001,Punjab,Islamabad,10011,both
4,1001100104.0,6,11,female,unmarried,1,5.0,2.0,5.0,urban,1,1001,Punjab,Islamabad,10011,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144396,4262002013.0,5,12,female,unmarried,2,NaN,NaN,13.0,rural,4,4262,Balochistan,Bolan/kachhi,42620,both
144397,4262002013.0,6,8,female,unmarried,2,NaN,NaN,13.0,rural,4,4262,Balochistan,Bolan/kachhi,42620,both
144398,4262002014.0,5,15,female,unmarried,2,NaN,NaN,13.0,rural,4,4262,Balochistan,Bolan/kachhi,42620,both
144399,4262002014.0,6,11,male,unmarried,2,NaN,NaN,4.0,rural,4,4262,Balochistan,Bolan/kachhi,42620,both


In [20]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.', index_label="id")